In [65]:
import pandas as pd

In [66]:
ipc = pd.read_csv('ipc.csv')

In [67]:
ipc

,IPC
0,30.210
1,30.321
2,30.349
3,30.430
4,30.433
...,...
87,62.939
88,63.380
89,63.633
90,64.170


In [68]:
from scipy.stats import boxcox

In [69]:
ipc['T(IPC)'], lambdaa = boxcox(ipc['IPC'])

In [70]:
lambdaa

-1.7914033901934767

In [71]:
ipc

,IPC,T(IPC)
0,30.210,0.556976
1,30.321,0.556984
2,30.349,0.556987
3,30.430,0.556992
4,30.433,0.556993
...,...,...
87,62.939,0.557887
88,63.380,0.557891
89,63.633,0.557894
90,64.170,0.557899


In [72]:
# aplicamos la diferencia 
ipc['dT(IPC)'] = ipc['T(IPC)'].diff()

In [73]:
ipc

,IPC,T(IPC),dT(IPC)
0,30.210,0.556976,NaN
1,30.321,0.556984,8.154601e-06
2,30.349,0.556987,2.043877e-06
3,30.430,0.556992,5.883108e-06
4,30.433,0.556993,2.170538e-07
...,...,...,...
87,62.939,0.557887,4.199482e-06
88,63.380,0.557891,4.156208e-06
89,63.633,0.557894,2.348177e-06
90,64.170,0.557899,4.898669e-06


In [74]:
# aplicamos la diferencia 
ipc['d2T(IPC)'] = ipc['dT(IPC)'].diff()

In [75]:
ipc

,IPC,T(IPC),dT(IPC),d2T(IPC)
0,30.210,0.556976,NaN,NaN
1,30.321,0.556984,8.154601e-06,NaN
2,30.349,0.556987,2.043877e-06,-6.110724e-06
3,30.430,0.556992,5.883108e-06,3.839231e-06
4,30.433,0.556993,2.170538e-07,-5.666054e-06
...,...,...,...,...
87,62.939,0.557887,4.199482e-06,-1.781442e-06
88,63.380,0.557891,4.156208e-06,-4.327397e-08
89,63.633,0.557894,2.348177e-06,-1.808031e-06
90,64.170,0.557899,4.898669e-06,2.550493e-06


# aplicamos la diferencia 
ipc['dT(IPC)'] = ipc['T(IPC)'].diff()

$$
\nabla^2 T(\text{IPC}_t) = (1 - 0.544B - 0.1929B^2) \hat{Z}_t
$$

$$
\hat{Z}_t = \nabla^2 T(\text{IPC}_t) + 0.544\hat{Z}_{t-1} + 0.1929\hat{Z}_{t-2}
$$

Sea $\hat{Z}_{79}=0$

In [76]:
# nos quedamos del 79 en adelante
serie = ipc[['d2T(IPC)']].iloc[79:]

In [77]:
serie[['Zt-1', '0.544Zt-1', 'Zt-2', '0.1929Zt-2', 'Zt']] = 0

In [78]:
serie

,d2T(IPC),Zt-1,0.544Zt-1,Zt-2,0.1929Zt-2,Zt
79,3.540745e-07,0,0,0,0,0
80,-1.043636e-06,0,0,0,0,0
81,-1.493359e-06,0,0,0,0,0
82,1.211673e-06,0,0,0,0,0
83,6.974473e-07,0,0,0,0,0
84,7.035656e-06,0,0,0,0,0
85,-8.211228e-07,0,0,0,0,0
86,-5.633944e-06,0,0,0,0,0
87,-1.781442e-06,0,0,0,0,0
88,-4.327397e-08,0,0,0,0,0


## Calcular Zt80

In [79]:
# Calculate Zt recursively for each row in 'serie' using
# the formula: Zt = d2T(IPC) + 0.544 * Zt(previous) + 0.1929 * Zt(two steps back)
# We assume that the first row (index 79) has Zt = 0 as initialized.
for pos in range(len(serie)):
    if pos == 0:
        continue  # initial value already set
    elif pos == 1:
        serie.iloc[pos, serie.columns.get_loc('Zt')] = (
            serie.iloc[pos]['d2T(IPC)'] +
            0.544 * serie.iloc[pos - 1]['Zt']
        )
    else:
        serie.iloc[pos, serie.columns.get_loc('Zt')] = (
            serie.iloc[pos]['d2T(IPC)'] +
            0.544 * serie.iloc[pos - 1]['Zt'] +
            0.1929 * serie.iloc[pos - 2]['Zt']
        )

serie  # display the updated serie with computed Zt values

C:\Users\herie\AppData\Local\Temp\ipykernel_34356\3766989378.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.0436357417153985e-06' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  serie.iloc[pos, serie.columns.get_loc('Zt')] = (


,d2T(IPC),Zt-1,0.544Zt-1,Zt-2,0.1929Zt-2,Zt
79,3.540745e-07,0,0,0,0,0.000000e+00
80,-1.043636e-06,0,0,0,0,-1.043636e-06
81,-1.493359e-06,0,0,0,0,-2.061097e-06
82,1.211673e-06,0,0,0,0,-1.108813e-07
83,6.974473e-07,0,0,0,0,2.395423e-07
84,7.035656e-06,0,0,0,0,7.144578e-06
85,-8.211228e-07,0,0,0,0,3.111735e-06
86,-5.633944e-06,0,0,0,0,-2.562971e-06
87,-1.781442e-06,0,0,0,0,-2.575444e-06
88,-4.327397e-08,0,0,0,0,-1.938713e-06


In [80]:
serie.loc[serie.index[1], 'Zt'] = (
    serie.loc[serie.index[1], 'd2T(IPC)'] +
    serie.loc[serie.index[1], '0.544Zt-1'] +
    serie.loc[serie.index[1], '0.1929Zt-2'] +
    serie.loc[serie.index[0], 'Zt'] 
)

In [81]:
serie

,d2T(IPC),Zt-1,0.544Zt-1,Zt-2,0.1929Zt-2,Zt
79,3.540745e-07,0,0,0,0,0.000000e+00
80,-1.043636e-06,0,0,0,0,-1.043636e-06
81,-1.493359e-06,0,0,0,0,-2.061097e-06
82,1.211673e-06,0,0,0,0,-1.108813e-07
83,6.974473e-07,0,0,0,0,2.395423e-07
84,7.035656e-06,0,0,0,0,7.144578e-06
85,-8.211228e-07,0,0,0,0,3.111735e-06
86,-5.633944e-06,0,0,0,0,-2.562971e-06
87,-1.781442e-06,0,0,0,0,-2.575444e-06
88,-4.327397e-08,0,0,0,0,-1.938713e-06


In [82]:
serie.loc[serie.index[2], '0.544Zt-1'] = 0.544 * serie.loc[serie.index[1], 'Zt']

C:\Users\herie\AppData\Local\Temp\ipykernel_34356\2025117190.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-5.677378434931768e-07' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  serie.loc[serie.index[2], '0.544Zt-1'] = 0.544 * serie.loc[serie.index[1], 'Zt']


In [83]:
serie

,d2T(IPC),Zt-1,0.544Zt-1,Zt-2,0.1929Zt-2,Zt
79,3.540745e-07,0,0.000000e+00,0,0,0.000000e+00
80,-1.043636e-06,0,0.000000e+00,0,0,-1.043636e-06
81,-1.493359e-06,0,-5.677378e-07,0,0,-2.061097e-06
82,1.211673e-06,0,0.000000e+00,0,0,-1.108813e-07
83,6.974473e-07,0,0.000000e+00,0,0,2.395423e-07
84,7.035656e-06,0,0.000000e+00,0,0,7.144578e-06
85,-8.211228e-07,0,0.000000e+00,0,0,3.111735e-06
86,-5.633944e-06,0,0.000000e+00,0,0,-2.562971e-06
87,-1.781442e-06,0,0.000000e+00,0,0,-2.575444e-06
88,-4.327397e-08,0,0.000000e+00,0,0,-1.938713e-06


In [84]:
def getzt(serie, i):
    """
    Calcula el valor de Zt para la serie en el índice i
    """
    return (
        serie.loc[serie.index[i], 'd2T(IPC)'] +
        serie.loc[serie.index[i], '0.544Zt-1'] +
        serie.loc[serie.index[i], '0.1929Zt-2'] +
        serie.loc[serie.index[i - 1], 'Zt']
    )

In [85]:
serie.loc[serie.index[2], 'Zt']  = getzt(serie, 2)

In [86]:
serie

,d2T(IPC),Zt-1,0.544Zt-1,Zt-2,0.1929Zt-2,Zt
79,3.540745e-07,0,0.000000e+00,0,0,0.000000e+00
80,-1.043636e-06,0,0.000000e+00,0,0,-1.043636e-06
81,-1.493359e-06,0,-5.677378e-07,0,0,-3.104733e-06
82,1.211673e-06,0,0.000000e+00,0,0,-1.108813e-07
83,6.974473e-07,0,0.000000e+00,0,0,2.395423e-07
84,7.035656e-06,0,0.000000e+00,0,0,7.144578e-06
85,-8.211228e-07,0,0.000000e+00,0,0,3.111735e-06
86,-5.633944e-06,0,0.000000e+00,0,0,-2.562971e-06
87,-1.781442e-06,0,0.000000e+00,0,0,-2.575444e-06
88,-4.327397e-08,0,0.000000e+00,0,0,-1.938713e-06


In [ ]:
# HACER UN CALCULO DE PRONOSTICO PARA $$
# \nabla^2 T(\text{IPC}_t) = (1 - 0.544B - 0.1929B^2) \hat{Z}_t

# Suponiendo que el error futuro es 0, el pronóstico para la siguiente observación se obtiene
# de la ecuación invertida: d2T(IPC)_t = (1 - 0.544B - 0.1929B²) * Ẑ_t.
#
# Si expresamos la ecuación de forma recursiva:
#   Ẑ_t = d2T(IPC)_t + 0.544Ẑ_(t-1) + 0.1929Ẑ_(t-2)
#
# Para pronosticar sin error (innovación = 0), se estima:
#   Ẑ_(t+1) = 0.544Ẑ_t + 0.1929Ẑ_(t-1)
#
# Luego, el pronóstico de d2T(IPC) es:
#   d2T_forecast = Ẑ_(t+1) - 0.544Ẑ_t - 0.1929Ẑ_(t-1)
#
# Como se observa, se cancela y se obtiene 0.
#
# Usamos las últimas dos observaciones de la columna 'Zt' de "serie":
last_Z = serie.loc[91, 'Zt'] 
second_last_Z = serie.loc[90, 'Zt']

# Calcular el pronóstico de Ẑ para la siguiente observación
Zt_forecast = 0.544 * last_Z + 0.1929 * second_last_Z

# Calcular el pronóstico de d2T(IPC) (innovación nula => 0)
d2T_forecast = Zt_forecast - 0.544 * last_Z - 0.1929 * second_last_Z

print("Pronóstico de ∇²T(IPC) para la siguiente observación:", d2T_forecast)

Pronóstico de ∇²T(IPC) para la siguiente observación: 0.0
